<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex09notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex09notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


---
## 統計的識別入門 (1)
---


---
### 決定論的識別 vs 統計的識別

ここでは「識別」の語を用いているが，**パターン認識** (**pattern recognition**) という語を用いることも多い（「統計的パターン認識」等）．
以下，識別対象のデータ/特徴量の値を $\mathbf{x}$ とし，$\mathbf{x}$ が所属するクラスを $y$ と表記する．

たとえば，$10\times 10$ 画素で1画素8bitのグレイスケール画像の識別を考えてみる．

画素数 $100$ だから，全ての可能な画像は全部で $256^{100} = 2^{800} = (2^{10})^{80}\sim 1000^{80} = 10^{240}$ 通り．これは，1の後ろに0が240個つづく，膨大な数（観測可能宇宙内の原子数は約 $10^{80}$と言われる）．
したがって，$\mathbf{x}$ とそれに対応した $y$ として起こり得る全ての値を記録した表を作って識別する… というのは無理．

そのため，何らかの方法で，$\mathbf{x}$ を入力すると $y$ の推定値を計算する仕組みを作ることが必要．この写像 $\mathbf{x}\rightarrow y$ を決定論的なものとするのが **決定論的識別**/**決定論的パターン認識** ，確率的なものとして扱うのが **統計的識別**/**統計的パターン認識**．

---
### 統計的識別の考え方

統計的識別では，特徴量 $\mathbf{x}$ もクラス $y$ も確率変数と考える．この授業では，

- $\mathbf{x}$ は連続型確率変数（より具体的には，実数値から成るベクトル），その確率密度関数は $p(\mathbf{x})$
- $y$ は離散型確率変数（$y \in \{猫,犬,カピバラ\}$みたいな），その生起確率は $p(y)$

とする．

- $p(\mathbf{x}, y)$: $\mathbf{x}, y$ の同時確率
- $p(y | \mathbf{x})$: クラス事後確率
- $p(y)$: クラス事前確率
- $p(\mathbf{x}|y)$: クラス条件付きの $\mathbf{x}$ の確率密度



ある特徴量 $\mathbf{x}$ に対して，

$$
p(猫|\mathbf{x}) = 0.2 \quad
p(犬|\mathbf{x}) = 0.75 \quad
p(カピバラ|\mathbf{x}) = 0.05
$$

のように事後確率が計算できれば，識別できそうだ．

統計的識別の考え方のひとつは，「与えられた $\mathbf{x}$ は事後確率 $p(y | \mathbf{x})$ が最大となるクラスに所属すると判定する」というルールを採用するもの．

$$
（\mathbf{x}のクラスの推定値）= \underset{y}{\operatorname{argmax}} p(y | \mathbf{x})
$$

※ 独立同一分布の仮定

この授業では，学習データの個々のペア $(\mathbf{x}_{n}, y_{n})$ は独立に同じ分布の同時確率 $p(\mathbf{x}, y)$ に従うと仮定する．このようなデータは **独立同一分布に従う**（**independently and identically disttibuted**, **i.i.d.**）という．

---
### 識別モデルと生成モデル

事後確率をどうやって推定しよう...

- **識別モデル**(**discriminative model**): $p(y|\mathbf{x})$ または識別境界を何らかの方法で直接推定する．例: ロジスティック回帰
- **生成モデル**(**generative model**): 以下で説明

#### 生成モデルとベイズの公式

**ベイズの公式**

$$
p(\mathbf{x}, y) = p(\mathbf{x}|y)p(y) = p(y|\mathbf{x})p(\mathbf{x})
$$

より，

$$
p(y|\mathbf{x}) = \frac{p(\mathbf{x}|y)p(y)}{p(\mathbf{x})}
$$

が成り立つ．この関係を用いて，事後確率のかわりに右辺をモデル化するのが生成モデルによるアプローチ．

特に，「事後確率最大のクラスに所属すると判定する」ルールの場合，

$$
%\begin{aligned}
\underset{y}{\operatorname{argmax}} p(y | \mathbf{x}) = \underset{y}{\operatorname{argmax}} \frac{p(\mathbf{x}|y)p(y)}{p(\mathbf{x})} = \underset{y}{\operatorname{argmax}} p(\mathbf{x}|y)p(y)
%\end{aligned}
$$

となる．$p(\mathbf{x})$ は $y$ によらないので，無視できる．
つまり，各クラスの事前確率 $p(y)$（どのクラスが出現しやすいか）と，クラスごとの特徴量の分布 $p(\mathbf{x}|y)$ （クラス $y$ の特徴量はどんな分布してるか）がわかれば，その積の大小を比較することでクラスを予測することができる．



$p(y)$ はどうやって推定する？

→ 学習データの出現頻度から推定するのが一般的．それがむずかしい場合は，全てのクラスで $p(y)$ が等しいとすることもある．



$p(\mathbf{x}|y)$ はどうやって推定する？

→ 学習データを用いて推定．

- パラメトリックモデル:  少数のパラメータを含む形で定式化された確率モデル
- ノンパラメトリックモデル:  （説明省略）

パラメトリックモデルのアプローチの典型例（というかもっとも単純な方法）は，クラスごとの分布 $p(\mathbf{x}|y)$ に正規分布をあてはめる，というもの．


---
### 1次元正規分布の最尤推定

#### 問題設定

データ: $\{ x_n \in \mathbb{R} \}_{n=1}^{N}$

が，ある平均・分散をもつ正規分布（ガウス分布, Gaussian distribution）から独立に生成されたと仮定（i.i.d.）．

モデル: $p(x; \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp{\left(-\frac{(x-\mu)^2}{2\sigma^2} \right)}$

をあてはめる → もっともあてはまりそうなパラメータ $\mu, \sigma^2$ を求める．



#### 最尤推定

あるデータ $x$ のモデルへのあてはまりの良さを「**尤度**(**likelihood**)」
$$
\ell(\mu, \sigma^2) = p(x; \mu, \sigma^2)
$$
で測る．このとき，データ全体の尤度は
$$
L(\mu, \sigma^2) = \prod_{n=1}^{N} p(x_n; \mu, \sigma^2) = \prod_{n=1}^{N} \ell(\mu, \sigma^2)
$$
とできる（i.i.d.だから）．この尤度が最大となるパラメータ $\mu, \sigma^2$ を求める．このようにしてパラメータにあてはまるモデルパラメータを推定することを，「**最尤推定**(**maximum likelihood estimation**)」という（ここでは1次元正規分布の場合で説明してるけど，一般のパラメトリックな確率モデルをあてはめる場合にも同様）．

実際に計算する際は，対数をとった「**対数尤度**(**log-likelihood**)」を最大化する方が扱いやすいことが多い．

$$
\log{L(\mu, \sigma^2)} = \sum_{n=1}^{N} \log{\ell(\mu, \sigma^2)}
$$

#### 対数尤度を最大にする

対数尤度を最大にする $\mu, \sigma^2$ の値（最尤推定の解） $\hat{\mu}, \hat{\sigma}^2$ を求めよう．

---
### 多次元正規分布の最尤推定

#### 問題設定

データ: $\{ \mathbf{x}_n \in \mathbb{R}^{D} \}_{n=1}^{N}$

が，ある平均・分散共分散行列をもつ正規分布（ガウス分布, Gaussian distribution）から独立に生成されたと仮定（i.i.d.）．

モデル: $p(\mathbf{x}; \mathbf{\mu}, \Sigma) = \frac{1}{\sqrt{(2\pi)^D|\Sigma|}} \exp{ \left( -\frac{1}{2} (\mathbf{x}-\mathbf{\mu})^{\top}\Sigma^{-1}(\mathbf{x}-\mathbf{\mu}) \right) }$

をあてはめる → もっともあてはまりそうなパラメータ $\mathbf{\mu}, \Sigma$ を求める．
$\mathbf{\mu}$ は $D$ 次元ベクトル, $\Sigma$ は $D \times D$ 対称行列．

#### 最尤推定の解

上記の問題において，対数尤度を最大化するパラメータ（最尤推定量 $\hat{\mathbf{\mu}}$ と $\hat{\Sigma}$）は次式のように与えられる（導出は省略）．

$$
\begin{aligned}
\hat{\mathbf{\mu}} &= \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n \\
\hat{\Sigma} &= \frac{1}{N}\sum_{n=1}^{N}(\mathbf{x}_n-\hat{\mathbf{\mu}})(\mathbf{x}_n-\hat{\mathbf{\mu}})^{\top}
\end{aligned}
$$

それぞれ，データの標本平均と標本分散共分散行列そのものに一致する．
